In [1]:
import json
import pandas as pd
import numpy as np
import netwulf as nw
import matplotlib.pyplot as plt
import networkx as nx
import random as random 
pd.set_option('display.max_colwidth', None)
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import seaborn as sns
from tqdm import tqdm

In [2]:
# read in the data
df = pd.read_csv('data/Game_of_Thrones_Script.csv')
df.shape
print(df.columns)

Index(['Release Date', 'Season', 'Episode', 'Episode Title', 'Name',
       'Sentence'],
      dtype='object')


In [3]:
# in season column remove the 'Season ' and convert to int
df['Season'] = df['Season'].str.replace('Season ', '').astype(int)

# in episode column remove the 'Episode ' and convert to int
df['Episode'] = df['Episode'].str.replace('Episode ', '').astype(int)

In [4]:
# tokenize the text and remove punctuation
stop_words = set(stopwords.words('english'))
# add some more words to the stop words
stop_words.update(['nt', 'l', 'know', 'would', 'wanted', 'want', 'like', 'yeah', 'oh', 'right', 'well', 'come', 'gonna', 'got', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'oh', 'ohh', 'ohhh', 'ohhhh', 'ohhhhh', 'ohhhhhh', 'ohhhhhhh', 'ohhhhhhhh', 'ohhhhhhhhh', 'ohhhhhhhhhh', 'ohhhhhhhhhhh', 'ohhhhhhhhhhhh', 'ohhhhhhhhhhhhh', 'ohhhhhhhhhhhhhh', 'ohhhhhhhhhhhhhhh', 'ohhhhhhhhhhhhhhhh', 'ohhhhhhhhhhhhhhhhh', 'ohhhhhhhhhhhhhhhhhh', 'ohhhhhhhhhhhhhhhhhhh', 'ohhhhhhhhhhhhhhhhh'])
lemmatizer = WordNetLemmatizer()
def tokenize(text):
    tokens = word_tokenize(text)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    words = [w for w in words if not w in stop_words]
    words = [lemmatizer.lemmatize(w) for w in words]

    # remove words with less than 3 characters
    words = [w for w in words if len(w) > 2]

    # remove common words like know and like
    
    return words

# tokenize the text
df['tokens'] = df['Sentence'].apply(tokenize)

# create new dataframe with season and episode
df_tokenized = df[['Season', 'Episode', 'Episode Title', 'Name', 'tokens']]

In [5]:
# create 1 long list of tokens for each episode
df_byepisode = df_tokenized.groupby(['Season', 'Episode', 'Episode Title'])['tokens'].apply(list).reset_index(name='tokens')
# convert list of lists to list
df_byepisode['tokens'] = df_byepisode['tokens'].apply(lambda x: [item for sublist in x for item in sublist])

# create 1 long list of tokens for each season
df_byseason = df_tokenized.groupby(['Season'])['tokens'].apply(list).reset_index(name='tokens')
# convert list of lists to list
df_byseason['tokens'] = df_byseason['tokens'].apply(lambda x: [item for sublist in x for item in sublist])

bipartite words.....